# TODO

Variables to explore:

- [x] PN17
- [x] PN25
- [ ] PN34
- [ ] PN35
- [ ] PN36
- [ ] PN38
- [ ] PN9

## Set up

In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from pathlib import Path
import polars as pl
import pandas as pd
import numpy as np
from rich import print as rprint

from pain.read import *
from pain.explore import *

In [ ]:
data_dir = Path("../data/raw")

In [ ]:
datasets = [
    Dataset("G214_PQ.sav", data_dir, "G214_PQ_", ["ID", "G214_PQ_PN17", "G214_PQ_PN25", "G214_PQ_PN34", "G214_PQ_PN35", "G214_PQ_PN36"]),
    Dataset("G214_SQ.sav", data_dir, "G214_SQ_", ["ID", "G214_SQ_PN17", "G214_SQ_PN25", "G214_SQ_PN34", "G214_SQ_PN35", "G214_SQ_PN36"]),
    Dataset("G217_PQ.sav", data_dir, "G217_PQ_", ["ID", "G217_PQ_PN17", "G217_PQ_PN25", "G217_PQ_PN34", "G217_PQ_PN35", "G217_PQ_PN36", "G217_PQ_PN38", "G217_PQ_PN9"]),
    Dataset("G217_SQ.sav", data_dir, "G217_SQ_", ["ID", "G217_SQ_PN17", "G217_SQ_PN25", "G217_SQ_PN34", "G217_SQ_PN35", "G217_SQ_PN36", "G217_SQ_PN38", "G217_SQ_PN9"])
]

In [ ]:
dataframes, metadata = read_and_filter_data(datasets)
df = combine_dataframes(dataframes)
meta = merge_dictionaries(metadata)

## PN17

In [ ]:
var = "PN17"

### Data

The existing data options for PN17 are all the same:
- 0: No
- 1: Yes
- 9: Missing

Changes to make:
- Convert 9 to -99 in all cases

In [ ]:
unique_values(df, var)

{'G214_PQ_PN17': (None, 0.0, 1.0, 9.0),
 'G214_SQ_PN17': (None, 0.0, 1.0, 9.0),
 'G217_PQ_PN17': (None, 0.0, 1.0, 9.0),
 'G217_SQ_PN17': (None, 0.0, 1.0, 9.0)}

### Metadata

The metadata is generally identical, with a minor discrepancy in the label for G217_SQ.
There are differences in value labels, but these values do not exist in the data, so they can be harmonised without changes to the raw data.

In [ ]:
m = filter_metadata(var, df, meta)
pd.DataFrame(m).T

,G214_PQ_PN17,G214_SQ_PN17,G217_PQ_PN17,G217_SQ_PN17
Label,Ever had back pain,Ever had back pain,Ever had back pain,Ever had back pain?
Field Type,Numeric,Numeric,Numeric,Numeric
Field Width,8,8,8,8
Decimals,0,0,0,0
Variable Type,scale,scale,scale,scale
Field Values,"{0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable',...","{0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable',...","{0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in inco...","{0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}"


## PN25

In [ ]:
var = "PN25"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_values(df, var)


{
    'G214_PQ_PN25': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN25': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN25': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN25': (None, 0.0, 1.0, 9.0)
}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN25 should be -88 (N/A)
- When PN17 is 1 (Yes), PN25 should be 0, 1, or -99
- When PN17 is 9 (Missing), PN25 should be 

In [ ]:
test = df.clone().collect()

In [ ]:
(test
 .filter(
     pl.col("G214_PQ_PN17") == 9
 ).select("G214_PQ_PN25")
 .unique()
 .to_dict(as_series=False)
)

{'G214_PQ_PN25': [9.0]}

In [ ]:
test.filter(test["G214_PQ_PN17"] == 0)

ID,G214_PQ_PN17,G214_PQ_PN25,G214_PQ_PN34,G214_PQ_PN35,G214_PQ_PN36,G214_SQ_PN17,G214_SQ_PN25,G214_SQ_PN34,G214_SQ_PN35,G214_SQ_PN36,G217_PQ_PN17,G217_PQ_PN9,G217_PQ_PN38,G217_PQ_PN25,G217_PQ_PN34,G217_PQ_PN35,G217_PQ_PN36,G217_SQ_PN17,G217_SQ_PN9,G217_SQ_PN38,G217_SQ_PN25,G217_SQ_PN34,G217_SQ_PN35,G217_SQ_PN36
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10010.0,0.0,8.0,8.0,8.0,8.0,1.0,0.0,0.0,8.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null
10090.0,0.0,8.0,8.0,8.0,8.0,1.0,1.0,0.0,1.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
10150.0,0.0,8.0,8.0,8.0,8.0,0.0,8.0,8.0,8.0,8.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
10170.0,0.0,8.0,8.0,8.0,8.0,1.0,1.0,1.0,1.0,1.0,0.0,7.0,7.0,7.0,7.0,7.0,7.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
10200.0,0.0,8.0,8.0,8.0,8.0,0.0,8.0,8.0,8.0,8.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
54200.0,0.0,8.0,8.0,8.0,8.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
54210.0,0.0,8.0,8.0,8.0,8.0,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null
54300.0,0.0,8.0,8.0,8.0,8.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null


### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN25': 'Seek pro advice for back pain',
        'G214_SQ_PN25': 'Seek pro advice for back pain',
        'G217_PQ_PN25': 'Ever sought health professional advice/treatment for back pain',
        'G217_SQ_PN25': 'Seek treatment for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN25': 'Numeric',
        'G214_SQ_PN25': 'Numeric',
        'G217_PQ_PN25': 'Numeric',
        'G217_SQ_PN25': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN25': 8, 'G214_SQ_PN25': 8, 'G217_PQ_PN25': 8, 'G217_SQ_PN25': 8},
    'Decimals': {'G214_PQ_PN25': '0', 'G214_SQ_PN25': '0', 'G217_PQ_PN25': '0', 'G217_SQ_PN25': '0'},
    'Variable Type': {
        'G214_PQ_PN25': 'scale',
        'G214_SQ_PN25': 'scale',
        'G217_PQ_PN25': 'scale',
        'G217_SQ_PN25': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN25': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN25 = Metadata(
    label= "Sought professional advice/treatment",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## PN34

In [ ]:
var = "PN34"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_values(df, var)

{'G214_PQ_PN25': (None, 0.0, 1.0, 8.0, 9.0),
 'G214_SQ_PN25': (None, 0.0, 1.0, 8.0, 9.0),
 'G217_PQ_PN25': (None, 0.0, 1.0, 7.0, 9.0),
 'G217_SQ_PN25': (None, 0.0, 1.0, 9.0)}

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN25': 'Seek pro advice for back pain',
        'G214_SQ_PN25': 'Seek pro advice for back pain',
        'G217_PQ_PN25': 'Ever sought health professional advice/treatment for back pain',
        'G217_SQ_PN25': 'Seek treatment for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN25': 'Numeric',
        'G214_SQ_PN25': 'Numeric',
        'G217_PQ_PN25': 'Numeric',
        'G217_SQ_PN25': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN25': 8, 'G214_SQ_PN25': 8, 'G217_PQ_PN25': 8, 'G217_SQ_PN25': 8},
    'Decimals': {'G214_PQ_PN25': '0', 'G214_SQ_PN25': '0', 'G217_PQ_PN25': '0', 'G217_SQ_PN25': '0'},
    'Variable Type': {
        'G214_PQ_PN25': 'scale',
        'G214_SQ_PN25': 'scale',
        'G217_PQ_PN25': 'scale',
        'G217_SQ_PN25': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN25': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN25 = Metadata(
    label= "Sought professional advice/treatment",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## Sandbox

What values exist in subsequent variables when PN17 (and for G217, PN9 and PN38) is 0?

In [ ]:
np.unique(
    df
    .filter(
        pl.col("G217_PQ_PN17") == 0,
        pl.col("G217_PQ_PN9") == 0,
        pl.col("G217_PQ_PN38") == 0
    )
    .select('G217_PQ_PN25', 'G217_PQ_PN34', 'G217_PQ_PN35', 'G217_PQ_PN36', 'G217_PQ_PN38', 'G217_PQ_PN9')
    .collect()
)#.to_pandas().value_counts()

array([0., 9.])

In [ ]:
(
    df
    .filter(
        pl.col("G217_SQ_PN17") == 0,
        pl.col("G217_SQ_PN9") == 0,
        pl.col("G217_SQ_PN38") == 0
    )
    .select("G217_SQ_PN17", 'G217_SQ_PN9', 'G217_SQ_PN38', 'G217_SQ_PN25', 'G217_SQ_PN34', 'G217_SQ_PN35', 'G217_SQ_PN36')
    # .unique()
)

G217_SQ_PN17,G217_SQ_PN9,G217_SQ_PN38,G217_SQ_PN25,G217_SQ_PN34,G217_SQ_PN35,G217_SQ_PN36
f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…
0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
(
    df
    .filter(
        pl.col("G217_PQ_PN17") == 0,
    )
    .select("G217_PQ_PN17", 'G217_PQ_PN9', 'G217_PQ_PN38', 'G217_PQ_PN25', 'G217_PQ_PN34', 'G217_PQ_PN35', 'G217_PQ_PN36')
    # .unique()
)

G217_PQ_PN17,G217_PQ_PN9,G217_PQ_PN38,G217_PQ_PN25,G217_PQ_PN34,G217_PQ_PN35,G217_PQ_PN36
f64,f64,f64,f64,f64,f64,f64
0.0,7.0,7.0,7.0,7.0,7.0,7.0
0.0,7.0,7.0,7.0,7.0,7.0,7.0
0.0,7.0,7.0,7.0,7.0,7.0,7.0
0.0,7.0,7.0,7.0,7.0,7.0,7.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…
0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0
